<a href="https://colab.research.google.com/github/djurjo/DABI/blob/main/PSO-project/example_numba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numba
from numba import jitclass, types, typed, njit, prange
from numba import int32, float32

import time
import random
import math

In [3]:
### A python function calculating things


def calculate():
  s = 0
  for x in range(10**5):
  
    s += math.tan(random.random())**2
  return s

@njit
def calculate_fast():
  s = 0
  for x in range(10**5):
  
    s += math.tan(random.random())**2
  return s


t1 = time.time()
x = calculate()
print("Calculate took: ", time.time() - t1, " seconds")

t1 = time.time()
x = calculate_fast()
print("Calculate_fast took: ", time.time() - t1, " seconds the first time.")

t1 = time.time()
x = calculate_fast()
print("Calculate_fast took: ", time.time() - t1, " seconds the second time")


Calculate took:  0.029013872146606445  seconds
Calculate_fast took:  0.09394407272338867  seconds the first time.
Calculate_fast took:  0.0047490596771240234  seconds the second time


In [14]:
### Lists are in general uglier...

ex_list = [random.random() for i in range(10**6)]

def calculate_list(ex_list):
  s = 0
  for x in ex_list:
    s += x**2 + math.sin(x)
  return x

t1 = time.time()
x = calculate_list(ex_list)
print("Calculate_list took ", time.time() - t1, " seconds")

Calculate_list took  0.17064404487609863  seconds


In [16]:
### If we make...
@njit  
def calculate_list_fast(ex_list):
  s = 0
  for x in ex_list:
    s += x**2 + math.sin(x)
  return x

t1 = time.time()
x = calculate_list_fast(ex_list)
print("Calculate_list_fast took ", time.time() - t1, " seconds")
t1 = time.time()
x = calculate_list_fast(ex_list)
print("Calculate_list_fast took ", time.time() - t1, " seconds the secon time")

/usr/local/lib/python3.6/dist-packages/numba/core/ir_utils.py:2031: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'ex_list' of function 'calculate_list_fast'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-16-e7f1e08c07c9>", line 3:
@njit  
def calculate_list_fast(ex_list):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


Calculate_list_fast took  1.7758708000183105  seconds
Calculate_list_fast took  1.5848565101623535  seconds the secon time


In [15]:
## The proper way is:

t_ex_list = typed.List()
[t_ex_list.append(x) for x in ex_list]

t1 = time.time()
x = calculate_list_fast(t_ex_list)
print("Calculate_list_fast took ", time.time() - t1, " seconds the first time")


t1 = time.time()
x = calculate_list_fast(t_ex_list)
print("Calculate_list_fast took ", time.time() - t1, " seconds the second time")


Calculate_list_fast took  0.09128975868225098  seconds the first time
Calculate_list_fast took  0.002692699432373047  seconds the second time


In [17]:
@njit(parallel = True)
def calculate_list_fast_par(ex_list):
  s = 0
  for i in prange(len(ex_list)):
    s += ex_list[i]**2 + math.sin(ex_list[i])
  return x
  
t_ex_list = typed.List()
[t_ex_list.append(x) for x in ex_list]

t1 = time.time()
x = calculate_list_fast_par(t_ex_list)
print("Calculate_list_fast took ", time.time() - t1, " seconds the first time")


t1 = time.time()
x = calculate_list_fast_par(t_ex_list)
print("Calculate_list_fast took ", time.time() - t1, " seconds the second time")

<ipython-input-17-0769f4abb51f>:5: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  s += ex_list[i]**2 + math.sin(ex_list[i])
/usr/local/lib/python3.6/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


Calculate_list_fast took  1.0873351097106934  seconds the first time
Calculate_list_fast took  0.04892110824584961  seconds the second time
